In [6]:
from huggingface_hub import login
import PyPDF2
from transformers import pipeline
from pathlib import Path
import os
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import json

In [ ]:
# Log in to Hugging Face
#login(token="") from env!

In [7]:
# Function to extract text from a single PDF file and save to a Markdown file
def extract_text_from_pdf():
    with open(pdf_path, "rb") as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for i, page in enumerate(reader.pages):
            page_text = page.extract_text()
            text += f"# Page {i + 1}\n\n" + page_text + "\n\n"
    return text

In [8]:
pdf_path = "/Users/alexpti4ka/Documents/Books/ВКР/logiko-semanticheskaya-model-informatsionno-spravochnyh-sistem-v-gosudarstvennom-upravlenii.pdf"
output_path = "/Users/alexpti4ka/Documents/Books/ВКР/Texts_from_pdf/extracted_text.txt"

In [9]:
if __name__ == "__main__":
    extracted_text = extract_text_from_pdf()
    with open(output_path, "w", encoding="utf-8") as md_file:
        md_file.write(extracted_text)
    print(f"Extracted text saved to {output_path}")

Extracted text saved to /Users/alexpti4ka/Documents/Books/ВКР/Texts_from_pdf/extracted_text.txt


In [ ]:
# pdf_path = "/Users/alexpti4ka/Documents/Books/ВКР/podhody-k-sozdaniyu-ontologiy-dlya-avtomatizirovannyh-sistem-v-mashinostroitelnyh-proizvodstvah.pdf"
# if __name__ == "__main__":
#     extracted_text = extract_text_from_pdf()
#     print(extracted_text[:1000])

In [4]:
device = 'cuda'  # or 'cpu' for translate on cpu
model_name = 'utrobinmv/t5_summary_en_ru_zh_base_2048'
model = TFT5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

/opt/anaconda3/lib/python3.11/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [20]:
# Function to summarize text using the model
def summarize_text(text, prefix="summary big: "):
    src_text = prefix + text
    input_ids = tokenizer(src_text, return_tensors="tf")['input_ids']
    generated_tokens = model.generate(input_ids, max_length=1000)
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return result[0]

In [9]:
# Read extracted text from file and summarize it
def summarize_from_file():
    with open(output_path, "r", encoding="utf-8") as file:
        document_text = file.read()
    summarized_text = summarize_text(document_text)
    print("Summarized Text:\n")
    print(summarized_text)

In [15]:
# Choose the type of summary: "summary: ", "summary brief: ", "summary big: "
summary_prefix = "summary big: "

In [21]:
summarize_from_file()

Summarized Text:

В 2020-305 годах разработчики машиностроительных предприятий, выпускающих наукоем-про-дукцию, представили проект онтологов для создания автоматических систем. В 2020-305 годах онтологии, разработанного на основе онтологов, и онтологов, которые, как предполагается, имеют нужную структуру, могут быть использованы для создания онтологов.


In [22]:
# Function to extract text from a single PDF file and split into paragraphs
def extract_text_to_json():
    with open(pdf_path, "rb") as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        paragraphs = []
        for page in reader.pages:
            page_text = page.extract_text()
            page_paragraphs = page_text.split("\n\n")  # Split by double newlines for paragraphs
            paragraphs.extend([para.strip() for para in page_paragraphs if para.strip()])
    with open(output_path, "w", encoding="utf-8") as json_file:
        json.dump(paragraphs, json_file, ensure_ascii=False, indent=4)
    return paragraphs

In [23]:
# Function to summarize a single paragraph using the model
def summarize_paragraph(paragraph, prefix=summary_prefix):
    src_text = prefix + paragraph
    input_ids = tokenizer(src_text, return_tensors="tf")['input_ids']
    generated_tokens = model.generate(input_ids, max_length=512)
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return result[0]

In [24]:
# Function to summarize all paragraphs from a JSON file
def summarize_from_json():
    with open(output_path, "r", encoding="utf-8") as json_file:
        paragraphs = json.load(json_file)
    summarized_paragraphs = [summarize_paragraph(para) for para in paragraphs]
    for i, summary in enumerate(summarized_paragraphs):
        print(f"Paragraph {i + 1} Summary:\n{summary}\n")

In [29]:
summarize_from_json()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/alexpti4ka/Documents/Books/ВКР/Texts_from_pdf/extracted_text.json'